# Link to CSV File: https://www.dropbox.com/s/ju3fval44resps5/stocks2000toPresent.csv?dl=0

## Pooling S&P Ticker Data from current listings on Wikipedia. The listed companies change over time. For now, we'll take whatever is listed and run it back to 2000. We'll lose a few, but should have enough interesting information to look at - for now.

In [1]:
!pip install pandas_datareader
!pip install yfinance 
!pip install yahoo_fin
import yahoo_fin.stock_info as si
import pandas as pd
from pandas_datareader import data as pdr
from google.colab import files
import numpy as np
import datetime 
import yfinance as yf
import io
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 5.2 MB/s 
     |████████████████████████████████| 83 kB 2.2 MB/s 
     |████████████████████████████████| 140 kB 53.9 MB/s 
     |████████████████████████████████| 106 kB 54.8 MB/s 
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24592 sha256=82c018d5e6a8b5eb6f74532467f03980fe424530a18a01862870ff1d11889d6f
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e


In [2]:
#wiki_source_link = https://en.wikipedia.org/wiki/List_of_S%26P_500_companies 

webpage = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [ ]:
type(webpage[0])

pandas.core.frame.DataFrame

In [ ]:
webpage[0].shape

(502, 9)

In [3]:
company_df = webpage[0]
company_df.head(5)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [4]:
company_df['Symbol'] = company_df['Symbol'].str.replace('.','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
company_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 502 non-null    object
 1   Security               502 non-null    object
 2   SEC filings            502 non-null    object
 3   GICS Sector            502 non-null    object
 4   GICS Sub-Industry      502 non-null    object
 5   Headquarters Location  502 non-null    object
 6   Date first added       457 non-null    object
 7   CIK                    502 non-null    int64 
 8   Founded                502 non-null    object
dtypes: int64(1), object(8)
memory usage: 35.4+ KB


In [5]:

print(company_df.Symbol.nunique())


503


In [6]:
## extracting sector firms
comp_energy = company_df[(company_df['GICS Sector'] == 'Energy')]
comp_materials = company_df[(company_df['GICS Sector'] == 'Materials')]
comp_industrials = company_df[(company_df['GICS Sector'] == 'Industrials')]
comp_consumer_discretionary = company_df[(company_df['GICS Sector'] == 'Consumer Discretionary')]
comp_consumer_staples = company_df[(company_df['GICS Sector'] == 'Consumer Staples')]
comp_healthcare = company_df[(company_df['GICS Sector'] == 'Health Care')]
comp_financials = company_df[(company_df['GICS Sector'] == 'Financials')]
comp_information_technology = company_df[(company_df['GICS Sector'] == 'Information Technology')]
comp_communication_services = company_df[(company_df['GICS Sector'] == 'Communication Services')]
comp_utilities = company_df[(company_df['GICS Sector'] == 'Utilities')]
comp_real_estate = company_df[(company_df['GICS Sector'] == 'Real Estate')]

In [7]:
start_date = datetime.datetime(2022,5,1)
print(start_date)
end_date = datetime.datetime.today()
print(end_date)

2022-05-01 00:00:00
2022-11-10 04:26:34.902428


In [8]:
def df_to_csv(sector):
  stock_price = pd.DataFrame()
  for i in sector.Symbol.unique():
    try:
      df = yf.download(i, start = start_date, end = end_date, interval = "1d")
      df['Ticker'] = i
      df['Sector'] = sector['GICS Sector'][sector['Symbol'] == i].values[0]
      df['Company'] = sector['Security'][sector['Symbol'] == i].values[0]
      stock_price = stock_price.append(df)

    except:
      print(df['Symbol'])
  
  return stock_price

In [9]:
a= df_to_csv(comp_energy)
b= df_to_csv(comp_materials)
c= df_to_csv(comp_industrials)
d= df_to_csv(comp_consumer_discretionary)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [11]:
e= df_to_csv(comp_consumer_staples) #flag lost BFB

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Got error from yahoo api for ticker BFB, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BFB: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%****

In [12]:
g= df_to_csv(comp_financials) #flag lost BRKB

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Got error from yahoo api for ticker BRKB, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BRKB: No data found for

In [13]:
f= df_to_csv(comp_healthcare)
h= df_to_csv(comp_information_technology)
i= df_to_csv(comp_communication_services)
j= df_to_csv(comp_utilities)
k= df_to_csv(comp_real_estate)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [26]:
df_big = pd.concat([a,b,c,d,e,f,g,h,i,j,k], ignore_index= False) 

In [28]:
df_big = df_big.reset_index()
df_big.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume              float64
Ticker               object
Sector               object
Company              object
dtype: object

In [29]:
df_big['Date'] = df_big['Date'].dt.strftime('%Y-%m-%d')

In [30]:
df_big

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Sector,Company
0,2022-05-02,40.310001,41.099998,39.180000,40.610001,40.223583,10677800.0,APA,Energy,APA Corporation
1,2022-05-03,40.639999,42.959999,40.580002,42.520000,42.115410,9096200.0,APA,Energy,APA Corporation
2,2022-05-04,43.980000,45.270000,42.209999,45.200001,44.769909,10228600.0,APA,Energy,APA Corporation
3,2022-05-05,44.830002,45.250000,40.680000,41.299999,40.907017,15617200.0,APA,Energy,APA Corporation
4,2022-05-06,41.980000,44.169998,41.130001,44.110001,43.690281,9499800.0,APA,Energy,APA Corporation
...,...,...,...,...,...,...,...,...,...,...
67129,2022-11-03,29.250000,29.760000,28.900000,29.389999,29.389999,2870400.0,WY,Real Estate,Weyerhaeuser
67130,2022-11-04,29.760000,30.299999,29.440001,30.280001,30.280001,2929300.0,WY,Real Estate,Weyerhaeuser
67131,2022-11-07,30.490000,31.040001,30.299999,30.959999,30.959999,2572300.0,WY,Real Estate,Weyerhaeuser
67132,2022-11-08,30.990000,31.180000,30.700001,31.020000,31.020000,2908300.0,WY,Real Estate,Weyerhaeuser


In [31]:
df_big.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
Ticker        object
Sector        object
Company       object
dtype: object

In [33]:
# create an empty list
l = []
ind = []

for i in df_big.index:
# create a dictionary with details
  dataDict = {'date': df_big.at[i, 'Date'],
           'minute': '9:30',
           'open':df_big.at[i, 'Open'],
           'high':df_big.at[i, 'High'],
           'low':df_big.at[i, 'Low'],
           'close':df_big.at[i, 'Close'],
           'volume':df_big.at[i, 'Volume']}

  ind.append(df_big.at[i, 'Ticker'])
  l.append(dataDict)



In [34]:
tidy_df = pd.DataFrame()
tidy_df['chart'] = l
tidy_df['ticker'] = ind

In [35]:
tidy_df

,chart,ticker
0,"{'date': '2022-05-02', 'minute': '9:30', 'open...",APA
1,"{'date': '2022-05-03', 'minute': '9:30', 'open...",APA
2,"{'date': '2022-05-04', 'minute': '9:30', 'open...",APA
3,"{'date': '2022-05-05', 'minute': '9:30', 'open...",APA
4,"{'date': '2022-05-06', 'minute': '9:30', 'open...",APA
...,...,...
67129,"{'date': '2022-11-03', 'minute': '9:30', 'open...",WY
67130,"{'date': '2022-11-04', 'minute': '9:30', 'open...",WY
67131,"{'date': '2022-11-07', 'minute': '9:30', 'open...",WY
67132,"{'date': '2022-11-08', 'minute': '9:30', 'open...",WY


In [36]:
#create new DataFrame by combining rows with same id values
df_new = tidy_df.groupby(['ticker'])['chart'].apply(list)

In [37]:
tidylist = tidy_df.ticker.unique()

In [38]:
tidylist.sort()

In [39]:
tidiestlist = tidylist.tolist()

In [43]:
final_df = pd.DataFrame()
final_df['ticker'] = tidiestlist
final_df['chart'] = tidiestlist

In [44]:
final_df

,ticker,chart
0,A,A
1,AAL,AAL
2,AAP,AAP
3,AAPL,AAPL
4,ABBV,ABBV
...,...,...
496,YUM,YUM
497,ZBH,ZBH
498,ZBRA,ZBRA
499,ZION,ZION


In [45]:
count = 0
# iterating through the elements of list
for i in tidiestlist:
    final_df.at[count, 'chart'] = df_new[i]
    count += 1


In [46]:
final_df.head()

,ticker,chart
0,A,"[{'date': '2022-05-02', 'minute': '9:30', 'ope..."
1,AAL,"[{'date': '2022-05-02', 'minute': '9:30', 'ope..."
2,AAP,"[{'date': '2022-05-02', 'minute': '9:30', 'ope..."
3,AAPL,"[{'date': '2022-05-02', 'minute': '9:30', 'ope..."
4,ABBV,"[{'date': '2022-05-02', 'minute': '9:30', 'ope..."


In [47]:
ticker = df_big['Ticker'].unique()

In [ ]:
pe = []
eps = []
beta = []
dividend = []
earnings = []
# Can get financials on each company, though creates a new dataframe for each company
income_statement = si.get_income_statement("aapl")

In [48]:
marketCap = []
for tick in ticker:
    try:
        marketCap.append(si.get_quote_table(tick)["Market Cap"])
    except:
        print('Error with: ', tick)

/usr/local/lib/python3.7/dist-packages/yahoo_fin/stock_info.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['ticker'] = ticker.upper()


In [81]:
df_small = df_big.drop_duplicates(subset = "Ticker", keep = 'first', inplace = False)
df_small = df_small.reset_index()

In [91]:
dfr = pd.DataFrame()

dfr['ticker'] = ticker
dfr['sector'] = df_small['Sector']
dfr['marketcap'] = marketCap
dfr['company'] = df_big['Company'].unique()

In [92]:
dfr

,ticker,sector,marketcap,company
0,APA,Energy,14.645B,APA Corporation
1,BKR,Energy,29.76B,Baker Hughes
2,CVX,Energy,341.448B,Chevron Corporation
3,COP,Energy,156.083B,ConocoPhillips
4,CTRA,Energy,21.084B,Coterra
...,...,...,...,...
496,VTR,Real Estate,16.666B,Ventas
497,VICI,Real Estate,19.831B,Vici Properties
498,VNO,Real Estate,4.313B,Vornado Realty Trust
499,WELL,Real Estate,28.946B,Welltower


In [106]:
final_df
REAL_final = pd.merge(final_df, dfr, on = "ticker", how = "inner")

In [107]:
REAL_final = REAL_final.dropna()

In [108]:
REAL_final

,ticker,chart,sector,marketcap,company
0,A,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Health Care,41.374B,Agilent Technologies
1,AAL,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Industrials,9.086B,American Airlines Group
2,AAP,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Consumer Discretionary,10.934B,Advance Auto Parts
3,AAPL,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Information Technology,2.213T,Apple Inc.
4,ABBV,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Health Care,260.974B,AbbVie
...,...,...,...,...,...
496,YUM,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Consumer Discretionary,35.777B,Yum! Brands
497,ZBH,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Health Care,22.096B,Zimmer Biomet
498,ZBRA,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Information Technology,12.125B,Zebra Technologies
499,ZION,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Financials,7.778B,Zions Bancorporation


In [109]:
d = {'M': '*1000000', 'B': '*1000000000', 'T': '*1000000000000'}

s_convert = REAL_final.marketcap.replace(d, regex=True).map(pd.eval)

In [110]:
REAL_final['marketcap'] = s_convert

In [103]:
REAL_final = REAL_final.set_index('ticker')

In [111]:
REAL_final

,ticker,chart,sector,marketcap,company
0,A,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Health Care,4.137400e+10,Agilent Technologies
1,AAL,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Industrials,9.086000e+09,American Airlines Group
2,AAP,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Consumer Discretionary,1.093400e+10,Advance Auto Parts
3,AAPL,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Information Technology,2.213000e+12,Apple Inc.
4,ABBV,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Health Care,2.609740e+11,AbbVie
...,...,...,...,...,...
496,YUM,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Consumer Discretionary,3.577700e+10,Yum! Brands
497,ZBH,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Health Care,2.209600e+10,Zimmer Biomet
498,ZBRA,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Information Technology,1.212500e+10,Zebra Technologies
499,ZION,"[{'date': '2022-05-02', 'minute': '9:30', 'ope...",Financials,7.778000e+09,Zions Bancorporation


In [113]:
REAL_final.to_json('fromMay2022.json', orient = 'records')
files.download('fromMay2022.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
uploaded = files.upload()
json_df = pd.read_json(io.StringIO(uploaded['data.json'].decode('utf-8')))

Saving data.json to data.json


In [ ]:
# feed date, minute, close price to each ticker chart column. It is a list of nested dictionaries. Column name is the key, value is data. Market cap should contain number.
# add open, high, low, and volume keys. 
json_df

,ticker,chart,sector,marketCap
0,SPY,"[{'date': '2021-12-31', 'minute': '09:30', 'ma...",Index,404954000000
1,MMM,"[{'date': '2021-12-31', 'minute': '09:30', 'ma...",Industrials,112481000000
2,ABT,"[{'date': '2021-12-31', 'minute': '09:30', 'ma...",Health Care,228587000000
3,ABBV,"[{'date': '2021-12-31', 'minute': '09:30', 'ma...",Health Care,197252000000
4,ABMD,"[{'date': '2021-12-31', 'minute': '09:30', 'ma...",Health Care,16912759037
...,...,...,...,...
482,ZBRA,"[{'date': '2021-12-31', 'minute': '09:30', 'ma...",Technology,31539792310
483,ZBH,"[{'date': '2021-12-31', 'minute': '09:30', 'ma...",Health Care,30645254383
484,ZION,"[{'date': '2021-12-31', 'minute': '09:30', 'ma...",Financials,9257464847
485,ZTS,"[{'date': '2021-12-31', 'minute': '09:30', 'ma...",Health Care,99381357830


In [ ]:
# Illustrating it with a graph
df = df_big[df_big['Symbol'] == 'META']
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.show()